In [1]:
import polars as pl
import pandas as pd
from psifr import fr
data_file = "2011_Polyn.csv"

In [2]:
raw = pd.read_csv(data_file)
data = fr.merge_free_recall(raw, list_keys=["task"], study_keys=["category", "period"])
data

,subject,list,item,input,output,study,recall,repeat,intrusion,task,category,period,prior_list,prior_input
0,21,1,BATTLESHIP,1.0,5.0,True,True,0,False,1,SHIPS,0.0,NaN,NaN
1,21,1,ROOT BEER,2.0,4.0,True,True,0,False,1,BEVERAGES,0.0,NaN,NaN
2,21,1,JOURNAL,3.0,2.0,True,True,0,False,1,READING MATERIALS,0.0,NaN,NaN
3,21,1,BLUEBERRY,4.0,NaN,True,False,0,False,1,FRUIT,0.0,NaN,NaN
4,21,1,GALE,5.0,3.0,True,True,0,False,1,WEATHER TERMS,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16894,45,28,BLUEGILL,21.0,3.0,True,True,0,False,1,FISH,0.0,NaN,NaN
16895,45,28,NINTENDO,22.0,4.0,True,True,0,False,1,TOYS,0.0,NaN,NaN
16896,45,28,LETTER,23.0,NaN,True,False,0,False,1,READING MATERIALS,0.0,NaN,NaN
16897,45,28,OSTRICH,24.0,5.0,True,True,0,False,1,BIRDS,0.0,NaN,NaN


In [3]:
df = (
    pl.DataFrame(data)
    .with_columns(
        pl.col(["input", "output", "period", "prior_list", "prior_input"]).cast(pl.Int64),
    )
    .with_columns(
        pl.col("period").replace(0, 1)
    )
    .drop("prior_list", "prior_input")
)
df.head()

subject,list,item,input,output,study,recall,repeat,intrusion,task,category,period
i64,i64,str,i64,i64,bool,bool,i64,bool,i64,str,i64
21,1,"""BATTLESHIP""",1,5,true,true,0,false,1,"""SHIPS""",1
21,1,"""ROOT BEER""",2,4,true,true,0,false,1,"""BEVERAGES""",1
21,1,"""JOURNAL""",3,2,true,true,0,false,1,"""READING MATERIALS""",1
21,1,"""BLUEBERRY""",4,null,true,false,0,false,1,"""FRUIT""",1
21,1,"""GALE""",5,3,true,true,0,false,1,"""WEATHER TERMS""",1


In [4]:
df.write_csv("2011_Polyn_scored.csv", null_value="n/a")